# Landsat upgrade

1. download sentinel/landsat
2. split into 400 pix by 400 pix tiles
3. use lat/lon bounds for each tile to split coincident landsat image 

In [ ]:
## libraries
import sys
import datacube
from odc.ui import with_ui_cbk

# plotting
from matplotlib import pyplot as plt
# maths
import numpy as np
# save png images
import imageio
import csv

## DEA scripts
sys.path.append("Scripts")

#from dea_datahandling import load_ard
from DEAPlotting import rgb
from multiprocessing import Pool
from IPython.display import clear_output


In [3]:
## Datacube connection
dc = datacube.Datacube(app="Sentinel_2")

In [4]:
# Function for making image in a loop

product="s2a_ard_granule"
bands = ["nbart_blue", "nbart_green", "nbart_red"]

csvfile = 'Points wgs rand 2k - Points wgs rand 2k.csv'
count=0
list_of_chips = []
x_space = -0.051
y_space = -0.034
maxcount = 20

with open(csvfile) as csv_file:
    readCSV = csv.reader(csv_file, delimiter=',')
    for row in readCSV:
        if row[7]=='1' and row[6]!='12':
            
            #print(row)
            x_min = float(row[3])
            y_min = float(row[4])
            x_max = float(row[3])-x_space
            y_max = float(row[4])+y_space
            uid = row[0]
            x=(x_min,x_max)
            y=(y_min,y_max)
            #xlist.append(x)
            #ylist.append(y)
            year = row[5]
            month = int(row[6])
            start_month = str(f"{month:02d}")
            end_month = str(f"{(month+1):02d}")
            start_date = (year+"-"+start_month)
            end_date = (year+"-"+end_month)
            date_range = start_date,end_date
            #my_tlist.append(date_range)
            
            

            
            #print(full_list)
            list_of_chips.append([uid,date_range,x,y])
            #print(uid,',')
            
            
            count+=1
            if count>maxcount:
                break
#print(tlist[0:3], xlist[0:3], ylist[0:3])

#idlist = range(len(my_tlist))

In [5]:
#list_of_chips
verbose=False

In [8]:

def chipper(chip):
    
    
#for chip in list_of_chips:


    idlist = chip[0]
    time = chip[1]
    x = chip[2]
    y = chip[3]
    
    # Create a query object
    if verbose:
        print(x)
        print(y)
        print(time)
    query = {
        "x": x,
        "y": y,
        "time": time,
        "output_crs": "EPSG:3577",
        "resolution": (-10, 10),
        "group_by": "solar_day",
    }
    
    ## download ard granule
    ds = dc.load(
        product=product,
        measurements = bands,
        progress_cbk = with_ui_cbk(),
        **query
    )
    if verbose:
        print(ds)
    if len(ds.sizes)==0:
        print("WARNING: NO DATA FOR TIME RANGE",time)
        #continue
        
    time_step_count=0
    if len(ds.sizes)!=0:
        for time_step in ds.time:
            
            img = np.moveaxis(ds[['nbart_red', 'nbart_green', 'nbart_blue']].isel(time=time_step_count).to_array().values,0,2)
            #print(img.shape)
            #np.flip(img,axis=0)# flip y axis
            # convert image to uint8 from uint16
            #print(np.min(img),np.max(img))
            img[img<0] = 0
            img = (img * (255./ 4000.)).astype(np.uint8)
            img[img>255] = 255
            ## Cut to 400x400
            n_y,n_x,_ = np.shape(img)

            dim_max=400
            assert n_x > dim_max, "ERR: X DIM IS < 400"
            assert n_y > dim_max, "ERR: Y DIM IS < 400"

            x_cut = float(dim_max)/n_x
            y_cut = float(dim_max)/n_y
            new_x_1 = x[0] + (x[1]-x[0])*x_cut



            new_y_1 = y[0] + (y[1]-y[0])*y_cut
            img_cut = img[:dim_max,:dim_max,:]

            if verbose:
                print(img.shape, img_cut.shape)
                print(x, '->', x[0], new_x_1)
                print(y, '->', y[0], new_y_1)

            #print(np.min(img),np.max(img))
            fname="sentinel_images/"+str(idlist)+"sentinel"+str(time_step_count)+".png"
            imageio.imwrite(fname,img_cut)
            if verbose:
                print("INFO: Saved figure: ",fname)
    
    
    ## download landsat equivalent
    query = {
        "x": (x[0], new_x_1),
        "y": (y[0], new_y_1),
        "time": time,
        "output_crs": "EPSG:3577",
        "resolution": (-25, 25),
        "group_by": "solar_day",
    }



    ## download ard granule
    #lsbands = ['Visible']
    ds2 = dc.load(
        product='ga_ls8c_ard_3',
        measurements = bands,
        progress_cbk = with_ui_cbk(),
        **query
    )
    
    if verbose:
        print(ds2)
    if len(ds2.sizes)==0:
        print("WARNING: NO DATA FOR TIME RANGE",time)
        #continue
        
    time_step_count=0
    if len(ds2.sizes)!=0:
        for time_step in ds2.time:
            lsimg = np.moveaxis(ds2[['nbart_red', 'nbart_green', 'nbart_blue']].isel(time=time_step_count).to_array().values,0,2)

            lsimg[lsimg<0] = 0
            lsimg = (lsimg * (255./ 4000.)).astype(np.uint8)
            lsimg[lsimg>255] = 255


            if verbose:
                print(lsimg.shape, lsimg.shape)


            lsfname="sentinel_images/"+str(idlist)+"land_sat_TS"+str(time_step_count)+".png"
            imageio.imwrite(lsfname,lsimg)
            time_step_count=+1
        if verbose:
            print("INFO: Saved figure: ",lsfname)
    clear_output(wait=True)
    
    

In [9]:
#np.moveaxis(ds2[['nbart_red', 'nbart_green', 'nbart_blue']].isel(time=2).to_array().values,0,2)
#time_steps = ds2.time
#print(ds2.time)
#for time_step in ds.time:
    #print(time_step)

In [10]:
pool = Pool(4)
pool.map(chipper, list_of_chips)
pool.close()
pool.join()

OperationalError: (psycopg2.OperationalError) SSL error: wrong version number

[SQL: SELECT agdc.dataset_type.id, agdc.dataset_type.name, agdc.dataset_type.metadata, agdc.dataset_type.metadata_type_ref, agdc.dataset_type.definition, agdc.dataset_type.added, agdc.dataset_type.added_by 
FROM agdc.dataset_type ORDER BY agdc.dataset_type.name ASC]

Process ForkPoolWorker-1:
Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/pytho